### First Assigment

### Another Exercise!

Let's code a shopping list!
We will have a list of elements like this:
    
x = sc.parallelize([["Apple",3,0.2],["Pear",5,0.35],["Milk",2,1.1],["Apple",3,0.2]])

Where the first element of each list is the product, the second the number of unit we bought and the third the unit price.

We want to have the list of how much we have spent in each product (ordered), and the total amount of money we have spent. 

(Optional) If we buy more than 10 products of the same type, we have a 10% discount of the final price

In [1]:
import pyspark
from operator import add
from pyspark import SparkConf
from pyspark import SparkContext

In [2]:
conf = SparkConf()
conf.setMaster("local[*]")
conf.setAppName("spark-basic")
sc = SparkContext(conf = conf)

In [2]:
#First Approach 

In [3]:
x = sc.parallelize([["Apple",3,0.2],["Pear",5,0.35],["Milk",2,1.1],["Apple",3,0.2]])

In [4]:
print(x)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274


In [5]:
x.take(4)

[['Apple', 3, 0.2], ['Pear', 5, 0.35], ['Milk', 2, 1.1], ['Apple', 3, 0.2]]

In [7]:
a = x.map(lambda x:x[0])
a.take(4)

['Apple', 'Pear', 'Milk', 'Apple']

In [9]:
f = x.map(lambda x: (x[0], x[1] * x[2]))
f.take(4)

[('Apple', 0.6000000000000001),
 ('Pear', 1.75),
 ('Milk', 2.2),
 ('Apple', 0.6000000000000001)]

In [15]:
g = f.reduceByKeyLocally(add)
g.items()

dict_items([('Apple', 1.2000000000000002), ('Pear', 1.75), ('Milk', 2.2)])

In [11]:
type(g)

dict_items

### Last one...

Replicate the last exercise, but the structure of the data is different. We have one object with products and prices. On the other hand, we have one list of the following form:

x = sc.parallelize([["Maria","Apple",1],["Maria","Pear",2],["Pau","Milk",4],["Laura","Apple",3]])

We want to know how much each of the have spent in total.

In [152]:
#['product',units, unit_price]
prodPrice = sc.parallelize([["Apple",3,0.2],["Pear",5,0.35],["Milk",2,1.1],["Apple",3,0.2]])

In [153]:
prodPrice.take(4)

[['Apple', 3, 0.2], ['Pear', 5, 0.35], ['Milk', 2, 1.1], ['Apple', 3, 0.2]]

In [154]:
prodPrice.keys().take(4)

['Apple', 'Pear', 'Milk', 'Apple']

In [155]:
#['client',product, units]
clientProdQty = sc.parallelize([["Maria","Apple",1],["Maria","Pear",2],["Pau","Milk",4],["Laura","Apple",3]])

In [156]:
clientProdQty.take(4)

[['Maria', 'Apple', 1],
 ['Maria', 'Pear', 2],
 ['Pau', 'Milk', 4],
 ['Laura', 'Apple', 3]]

In [157]:
clientProdQty.keys().take(4)

['Maria', 'Maria', 'Pau', 'Laura']

In [158]:
clientProdQty_= clientProdQty.map(lambda x: [x[1], (x[0], x[2])])
clientProdQty_.take(4)

[['Apple', ('Maria', 1)],
 ['Pear', ('Maria', 2)],
 ['Milk', ('Pau', 4)],
 ['Apple', ('Laura', 3)]]

In [159]:
prodPrice_ = prodPrice.map(lambda x: [x[0],x[2]])
prodPrice_.take(4)

[['Apple', 0.2], ['Pear', 0.35], ['Milk', 1.1], ['Apple', 0.2]]

In [160]:
prodPrice_ = prodPrice_.groupByKey()
prodPrice_.collect()

[('Apple', <pyspark.resultiterable.ResultIterable at 0x7f8ad3e40b20>),
 ('Milk', <pyspark.resultiterable.ResultIterable at 0x7f8ad868a0a0>),
 ('Pear', <pyspark.resultiterable.ResultIterable at 0x7f8ad868a340>)]

In [161]:
prodPrice_ = prodPrice_.map(lambda x: [x[0], min(x[1])])
prodPrice_.take(4)

[['Apple', 0.2], ['Milk', 1.1], ['Pear', 0.35]]

In [162]:
#prodPrice_.join(clientProdQty_).collect()

In [164]:
prdCliQtyPrice = clientProdQty_.join(prodPrice_)

In [165]:
prdCliQtyPrice.take(4)

[('Milk', (('Pau', 4), 1.1)),
 ('Pear', (('Maria', 2), 0.35)),
 ('Apple', (('Maria', 1), 0.2)),
 ('Apple', (('Laura', 3), 0.2))]

In [192]:
prdCliQtyPrice_ = prdCliQtyPrice.map(lambda x: ( x[1][0][0], round((x[1][0][1] * x[1][1]),2), x[0] ) )
prdCliQtyPrice_.take(4)

[('Pau', 4.4, 'Milk'),
 ('Maria', 0.7, 'Pear'),
 ('Maria', 0.2, 'Apple'),
 ('Laura', 0.6, 'Apple')]

In [193]:
prdCliQtyPrice_ = prdCliQtyPrice_.map(lambda x: (x[0], x[1]) )
prdCliQtyPrice_.take(4)

[('Pau', 4.4), ('Maria', 0.7), ('Maria', 0.2), ('Laura', 0.6)]

In [194]:
ClientTotSpent = prdCliQtyPrice_.reduceByKeyLocally(add)
ClientTotSpent.items()

dict_items([('Pau', 4.4), ('Maria', 0.8999999999999999), ('Laura', 0.6)])